In [14]:
import scipy.io as sio
import numpy as np
import emd


In [38]:
signal=sio.loadmat(r"E:\Donders\0_rem_test\raw\CN\Post_trial5_2018-02-22_13-22-10\post_trial5_HPC_merged.mat")['HPC']
states=sio.loadmat(r"E:\Donders\0_rem_test\raw\CN\Post_trial5_2018-02-22_13-22-10\post_trial5_states_merged.mat")['states']

In [6]:
def get_rem_states(states, sample_rate):
    try:
        states = np.squeeze(states)
        rem_state_indices = np.where(states == 5)[0]
        
        # Check if there are any REM states
        if len(rem_state_indices) == 0:
            return np.array([])

        rem_state_changes = np.diff(rem_state_indices)
        split_indices = np.where(rem_state_changes != 1)[0] + 1
        split_indices = np.concatenate(([0], split_indices, [len(rem_state_indices)]))
        consecutive_rem_states = np.empty((len(split_indices) - 1, 2))
        for i, (start, end) in enumerate(zip(split_indices, split_indices[1:])):
            start = rem_state_indices[start] * int(sample_rate)
            end = rem_state_indices[end - 1] * int(sample_rate)
            consecutive_rem_states[i] = np.array([start, end])
        consecutive_rem_states = np.array(consecutive_rem_states)
        null_states_mask = np.squeeze(np.diff(consecutive_rem_states) > 0)
        consecutive_rem_states = consecutive_rem_states[null_states_mask]
        return consecutive_rem_states
    except IndexError as e:
        print(f"An IndexError occurred in get_rem_states: {e}")
        return np.array([])  # or any default value you want
    

In [39]:
rem_state_8 = get_rem_states(states, 2500)[7].astype(int)

In [40]:
print(rem_state_8.shape)

(2,)


In [41]:
x=np.squeeze(signal[rem_state_8[0]:rem_state_8[1]])

In [42]:
print(x.shape)

(180000,)


In [44]:
y=emd.sift.iterated_mask_sift(x,mask_0='zc',sample_rate=2500,ret_mask_freq=True)

In [19]:
def tg_split(mask_freq, theta_range=(5, 12)):
    """
        Split a frequency vector into sub-theta, theta, and supra-theta components.

        Parameters:
        mask_freq (numpy.ndarray): A frequency vector or array of frequency values.
        theta_range (tuple, optional): A tuple defining the theta frequency range (lower, upper).
            Default is (5, 12).

        Returns:
        tuple: A tuple containing boolean masks for sub-theta, theta, and supra-theta frequency components.

        Notes: - This function splits a frequency mask into three components based on a specified theta frequency
        range. - The theta frequency range is defined by the 'theta_range' parameter. - The resulting masks 'sub',
        'theta', and 'supra' represent sub-theta, theta, and supra-theta frequency components.
    """
    lower = np.min(theta_range)
    upper = np.max(theta_range)
    mask_index = np.logical_and(mask_freq >= lower, mask_freq < upper)
    sub_mask_index = mask_freq < lower
    supra_mask_index = mask_freq > upper
    sub = sub_mask_index
    theta = mask_index
    supra = supra_mask_index

    return sub, theta, supra

In [33]:
def extrema(x):
    """
    Find extrema (peaks, troughs) and zero crossings in a 1D signal.

    Parameters:
    x (numpy.ndarray): The input 1D signal.

    Returns:
    tuple: A tuple containing:
        - numpy.ndarray: Indices of zero-crossings in the input signal.
        - numpy.ndarray: Indices of troughs in the input signal.
        - numpy.ndarray: Indices of peaks in the input signal.

    Notes:
    - This function identifies and returns the indices of zero-crossings, troughs, and peaks in a given 1D signal.
    - Zero-crossings are points where the signal crosses the zero axis.
    - Troughs are local minima, and peaks are local maxima in the signal.
    """
    zero_xs = zero_cross(x)
    peaks = np.empty((0,)).astype(int)
    troughs = np.empty((0,)).astype(int)
    for t1, t2 in zip(zero_xs, zero_xs[1:]):
        extrema0 = np.argmax(np.abs(x[t1:t2])).astype(int) + t1
        if bool(x[extrema0] > 0):
            peaks = np.append(peaks, extrema0)
        else:
            troughs = np.append(troughs, extrema0)
    return zero_xs, troughs, peaks
def zero_cross(x):
    """
    Find the indices of zero-crossings in a 1D signal.

    Parameters:
    x (numpy.ndarray): The input 1D signal.

    Returns:
    numpy.ndarray: An array of indices where zero-crossings occur in the input signal.

    Notes:
    - This function identifies the indices where zero-crossings occur in a given 1D signal.
    - It detects both rising and falling zero-crossings.
    """
    decay = np.logical_and((x > 0)[1:], ~(x > 0)[:-1]).nonzero()[0]
    rise = np.logical_and((x <= 0)[1:], ~(x <= 0)[:-1]).nonzero()[0]
    zero_xs = np.sort(np.append(rise, decay))
    return zero_xs

In [45]:
print(y[1])

[0.15888998 0.01026717 0.00600468 0.00269528 0.00082731 0.00028074]


In [46]:
_,theta,_=tg_split(y[1],theta_range=(5, 12))

In [47]:
print(theta)

[False False False False False False]


In [48]:
IMF=y[0].T
theta_IMF=IMF[theta]


In [49]:
print(theta_IMF.shape)

(0, 180000)


In [50]:
theta_signal=np.sum(theta_IMF,axis=0) 

In [51]:
print(theta_signal.shape)

(180000,)


In [52]:
zero_x=zero_cross(theta_signal)

In [53]:
print(zero_x.shape)

(0,)


In [54]:
zero_x, troughs, peaks=extrema(theta_signal)

In [55]:
print(zero_x.shape)
print(troughs.shape)
print(peaks.shape)

(0,)
(0,)
(0,)
